In [23]:
import pandas as pd
from openai import AzureOpenAI
import numpy as np

In [24]:
#%%% JSON content script
import pandas as pd
import json
from sqlalchemy import create_engine

# Make a connection to the database of Netivity / Optimizers using SQLAlchemy
connection_string = 'mssql+pyodbc:///?odbc_connect=' \
                    'Driver%3D%7BODBC+Driver+17+for+SQL+Server%7D%3B' \
                    'Server%3Dsql-maxaro-prod.database.windows.net%3B' \
                    'Database%3Dsqldb-maxaro-prod%3B' \
                    'UID%3Dbiadmin@maxaro.nl%3B' \
                    'Authentication%3DActiveDirectoryInteractive'

engine = create_engine(connection_string)

# Create the SQL-query to retrieve the "EmailID," "To," and "Content" columns from the "Emails" table
sql = """SELECT
A.ProductId, A.ArticleNumber, A.Name,

CategoryProduct.Parent_Category AS 'Parent Category', CategoryProduct.Category AS Category,

Benefits.Benefits_1 AS Productvoordelen_1, Benefits.Benefits_2 AS Productvoordelen_2, Benefits.Benefits_3 AS Productvoordelen_3, Benefits.Benefits_4 AS Productvoordelen_4, Benefits.Benefits_5 AS Productvoordelen_5,

USP_codes.USP_1, USP_codes.USP_2, USP_codes.USP_3,

B3.Value AS Productomschrijving, 
B5.Value AS WarningText,
B6.Value AS WarningUrgentText,
B7.Value AS ExtraInformation,
B8.Value AS 'Standaard meegeleverd',
V.Name AS 'Product Variant'

FROM Products AS A

LEFT JOIN (SELECT Cate_Group.ProductID, Cate.Parent_Category, Cate.Category
FROM [dbo].[CategoryProducts] AS Cate_Group
LEFT JOIN (SELECT B.Name As Parent_Category, Categories.Name AS Category, Categories.CategoryId
FROM Categories AS Categories
LEFT JOIN Categories AS B
ON Categories.ParentCategoryId = B.CategoryId) AS Cate
ON Cate_Group.categoryID = Cate.CategoryId
WHERE Cate_Group.IsMainCategoryForProduct = 1) AS CategoryProduct
ON A.ProductId = CategoryProduct.ProductId

LEFT JOIN(SELECT USP.ProductId, Y1.Value AS Benefits_1, Y2.Value AS Benefits_2, Y3.Value AS Benefits_3, Y4.Value AS Benefits_4, Y5.Value AS Benefits_5
FROM
(SELECT productID,
    MAX(CASE WHEN Sequence = 0 THEN textlocalizedtextid END) AS Sequence0,
    MAX(CASE WHEN Sequence = 1 THEN textlocalizedtextid END) AS Sequence1,
    MAX(CASE WHEN Sequence = 2 THEN textlocalizedtextid END) AS Sequence2,
	MAX(CASE WHEN Sequence = 3 THEN textlocalizedtextid END) AS Sequence3,
	MAX(CASE WHEN Sequence = 4 THEN textlocalizedtextid END) AS Sequence4
FROM [dbo].[ProductBenefits]
GROUP BY productID) AS USP
LEFT JOIN LocalizedTextValues AS Y1
ON USP.Sequence0 = Y1.LocalizedTextId
LEFT JOIN LocalizedTextValues AS Y2
ON USP.Sequence1 = Y2.LocalizedTextId
LEFT JOIN LocalizedTextValues AS Y3
ON USP.Sequence2 = Y3.LocalizedTextId
LEFT JOIN LocalizedTextValues AS Y4
ON USP.Sequence3 = Y4.LocalizedTextId
LEFT JOIN LocalizedTextValues AS Y5
ON USP.Sequence4 = Y5.LocalizedTextId) AS Benefits
ON A.ProductId = Benefits.ProductID

LEFT JOIN (SELECT USP.ProductId, X1.Value AS USP_1, X2.Value AS USP_2, X3.Value AS USP_3
FROM(
SELECT productID,
    MAX(CASE WHEN Sequence = 0 THEN TextLocalizedTextId END) AS Sequence0,
    MAX(CASE WHEN Sequence = 1 THEN TextLocalizedTextId END) AS Sequence1,
    MAX(CASE WHEN Sequence = 2 THEN TextLocalizedTextId END) AS Sequence2
FROM ProductUsps
GROUP BY productID) AS USP
LEFT JOIN LocalizedTextValues AS X1
ON USP.Sequence0 = X1.LocalizedTextId
LEFT JOIN LocalizedTextValues AS X2
ON USP.Sequence1 = X2.LocalizedTextId
LEFT JOIN LocalizedTextValues AS X3
ON USP.Sequence2 = X3.LocalizedTextId) AS USP_codes
ON A.ProductId = USP_codes.ProductId

LEFT JOIN ProductVariants AS V
ON A.ProductVariantId = V.ProductVariantId
LEFT JOIN LocalizedTextValues AS B3
ON A.DescriptionLocalizedTextId = B3.LocalizedTextId
LEFT JOIN LocalizedTextValues AS B5
ON A.WarningTextLocalizedTextId = B5.LocalizedTextId
LEFT JOIN LocalizedTextValues AS B6
ON A.WarningUrgentTextLocalizedTextId = B6.LocalizedTextId
LEFT JOIN LocalizedTextValues AS B7
ON A.ExtraInformationLocalizedTextId = B7.LocalizedTextId
LEFT JOIN LocalizedTextValues AS B8
ON A.DefaultIncludedLocalizedTextId = B8.LocalizedTextId
"""

# Get the email data as a Pandas DataFrame
json_data = pd.read_sql(sql, engine)

# Close the database connection
engine.dispose()


In [25]:
prodDes = json_data[(json_data['Productomschrijving'].isna()) & (json_data['Parent Category'] != 'Inactieve producten') & (json_data['Parent Category'] != 'Toebehoren')]

In [26]:
mask = prodDes[['Productvoordelen_1', 'Productvoordelen_2', 'Productvoordelen_3', 'Productvoordelen_4', 'Productvoordelen_5']].dropna(thresh=2).index

# Use the mask to filter the entire prodDes DataFrame
prodDes_filtered = prodDes.loc[mask]

In [27]:
engine = create_engine(connection_string)

# Construct your SQL query. This is just an example.
query = f"""SELECT A.ProductId, P.ArticleNumber, P.Name, A.Property, B.Value
        FROM ProductDynamicPropertyValues AS A
        LEFT JOIN LocalizedTextValues AS B
        ON A.ValueLocalizedTextId = B.LocalizedTextId
        LEFT JOIN Products AS P
        ON A.ProductId = P.ProductId"""

try:
    specs = pd.read_sql(query, engine)

finally:
    engine.dispose()

In [29]:
def concatenate_properties(series):
    return ', '.join(series.dropna().unique())

specs['Specifications'] = specs['Property'].astype(str) + ': ' + specs['Value'].astype(str)
grouped_specs = specs.groupby(['ProductId', 'ArticleNumber', 'Name']).agg({
    'Specifications': concatenate_properties
}).reset_index()

# Merge with prodDes_filtered
result_df = pd.merge(prodDes_filtered, grouped_specs, on=['ProductId', 'ArticleNumber', 'Name'], how='inner')


In [30]:
client = AzureOpenAI(
    api_key = "54a267e072934050a8df635e4f6da7b5",
    api_version = "2023-05-15",
    azure_endpoint = "https://maxbotai.openai.azure.com/"
    )

In [31]:
df_prodDes = pd.DataFrame(columns=['Product', 'Description'])
new_row = {'Product': '', 'Description': ''}
for index, row in result_df.iterrows():
    text = row['Name'] + ' ' + row['Specifications'] + ' ' + row['Productvoordelen_1'] + ' ' + row['Productvoordelen_2'] + ' ' + row['Productvoordelen_3'] + ' ' + row['Productvoordelen_4'] + ' ' + row['Productvoordelen_5']

    response = client.chat.completions.create(
        model="gpt-35-turbo",
        messages = [
            {"role": "system", "content": "Maak een overtuigende en positieve productbeschrijving voor het volgende artikel. Benadruk de belangrijkste kenmerken, voordelen en onderscheidende eigenschappen. Gebruik duidelijke en begrijpelijke taal om de lezer te boeien. Stel je voor dat je tegen een potentiële klant spreekt die op zoek is naar de beste eigenschappen van het product. Maak de beschrijving ongeveer 150-200 woorden. Verder moet de tekst seo-geoptimaliseerd zijn, dus let op: Schrijf voor kopers, niet voor bots; Benadruk voordelen, voeg functies toe; Richt je op de juiste SEO-productzoekwoorden; Plaats zoekwoorden strategisch in je tekst; Laat de lengte afhangen van de bekendheid van de koper; Maak een duidelijke call-to-action; Maak unieke productbeschrijvingen voor elk POP. Tot slot en erg belangrijk: als het product erg lijkt op een eerder product dat u had, bijvoorbeeld alleen een andere kleur of maat, hergebruik dan dezelfde beschrijving en verander alleen de details."},
            {"role": "user", "content": f"Dit is het product waarvoor we een beschrijving nodig hebben: {text}, en dit is de eerder gegenereerde beschrijving: {new_row['Description']}"},
        ])
    new_row = {'Product': row['Name'], 'Description': response.choices[0].message.content}
    df_prodDes.loc[len(df_prodDes)] = new_row

KeyboardInterrupt: 

In [ ]:
#df_prodDes.to_excel('train_data.xlsx', index=False)